# Extração, transformação e carga de dados

O processo de ETL é uma parte fundamental do trabalho com dados e consiste em três etapas:

- **Extração**: a coleta de dados, potencialmente a partir de múltiplas fontes heterogêneas. Pode envolver raspagem de páginas web, acesso a interfaces de programação (APIs) ou consultas a bancos de dados.
- **Transformação**: a reorganização dos dados, envolvendo operações como união, cruzamento e agregação.
- **Carga**: a persistência do novo conjunto de dados onde se quer armazená-lo.

Este notebook foca em exemplos de métodos de transformação com o Pandas.

Para isso usaremos três dataframes artificiais em nossos exemplos: `df_a`, `df_b` e `df_c`.

## Criando dataframes a partir de dicionários

Como vamos criar dataframes customizados para nossos exemplos, precisaremos do auxílio de **dicionários**.

Um dicionário é um tipo de objeto Python que permite armazenar valores indexados por chaves, similar ao que o `DataFrame` do Pandas faz.

Usamos a notação abaixo para criar um dicionário:

```python3
nome = {
        chave1: valor1,
        chave2: valor2,
        ...
        chaveN: valorN
        }
```

Acessamos um valor em um dicionário através da sua chave, usando a notação `dicionário[chave]`.

No exemplo a seguir, o dicionário `dados_df_a` têm como chaves os nomes das séries associadas:

In [ ]:
import pandas as pd

In [ ]:
dados_df_a = {
            'id_indivíduo': ['1', '2', '3', '4', '5'],
            'nome': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'], 
            'sobrenome': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']
            }

Note que cada série é representada como uma lista.

Criar um `DataFrame` a partir de um dicionário é bem simples:

In [ ]:
df_a = pd.DataFrame(dados_df_a)
df_a

Seguindo o mesmo modelo, vamos criar o dataframe `df_b`:

In [ ]:
dados_df_b = {
            'id_indivíduo': ['4', '5', '6', '7', '8'],
            'nome': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
            'sobrenome': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']
            }

In [ ]:
df_b = pd.DataFrame(dados_df_b)
df_b

In [ ]:
dados_df_c = {
            'id_indivíduo': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
            'id_exame': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]
            }

In [ ]:
df_c = pd.DataFrame(dados_df_c)
df_c

## União de dados

Uma das operações comuns é unir observações que apresentam as mesmas características, mas estão em diferentes dataframes. 

Para isso usaremos o comando `concat` que recebe uma lista com ***n*** objetos `DataFrame` como parâmetro.

In [ ]:
df_new = pd.concat([df_a, df_b])
df_new

Também seria possíve unir objetos `DataFrame` com características distintas.

No entanto, essa operação produziria um `DataFrame` com muitos dados faltando:

In [ ]:
pd.concat([df_a, df_c])

## Cruzando dados

No exemplo anterior, vimos o resultado de unir dataframes cujas características não são idênticas.

No entanto, quanto temos pelo menos uma característica em comum entre dois dataframes, podemos **cruzar**
 esses dados, produzindo um novo dataframe que reúne toda a informação dos dataframes originais.

No exemplo, abaixo as observações do dataframe **à esquerda** (`df_a`) e do dataframe **à direita** (`df_c`) foram cruzadas, tomando como característica em comum `id_indivíduo`. 

Como você pode ver, o novo dataframe reúne as informações de ambos os dataframes usados no cruzamento dos dados:

In [ ]:
pd.merge(df_a, df_c, on='id_indivíduo')

Em algumas situações, a mesma característica pode estar representada por diferentes nomes nos dataframes que se deseja cruzar.

Nesses casos, podemos usar os argumentos `left_on` e `right_on` para especificar, respectivamente, os nomes da característica no dataframe à esquerda e no dataframe à direita.

### Tipos de cruzamento

Uma operação de cruzamento de dados combina dados de dois dataframes que apresentem uma característica em comum.

No exemplo anterior, a característica em comum era o campo `id_indivíduo`.

Note que as observações presentes no dataframe `df_c` cujos valores para `id_indivíduo` não estão presentes no dataframe `df_a` não foram mostradas.

Se quisermos que essas observações sejam preservadas, podemos usar um **cruzamento à direita**.

In [ ]:
pd.merge(df_a, df_c, on='id_indivíduo', how='right')

O resultado acima mostra tanto as observações com `id_indivíduo` presentes nos dois dataframes como o restante das observações do dataframe à direita. 

Note que as observações adicionadas pelo cruzamento à direita apresentam dados faltando.

O mesmo aconteceria se usássemos um **cruzamento à esquerda**:

In [ ]:
pd.merge(df_b, df_c, on='id_indivíduo', how='left')

Nesse caso, a observação do dataframe `df_b` cujo `id_indivíduo` não estava presente no dataframe `df_c` foi mantida.

Em um caso mais extremo, podemos usar um **cruzamento externo**, que mantém todas as observações de ambos os dataframes:

In [ ]:
pd.merge(df_b, df_c, on='id_indivíduo', how='outer')

## Agregando dados

As operações de união e cruzamento tem por objetivo reunir informações espalhadas em múltiplas bases em um único dataframe.

Um tipo complementar de operação é a **agregação**, que visa resumir blocos de informações através de estatísticas descritivas. 

As principais formas de agregação são obtidas por meio de pivoteamento, seja unidimensional (**grupos**) ou bidimensional (**tabelas dinâmicas**).  

### Grupos

Organizar os dados em grupos pode ser útil tanto para analisar cada grupo como para calcular estatísticas por grupo.

O primeiro passo da agregação é definir uma ou mais características usadas como fatores do agrupamento.

No exemplo abaixo, agrupamos os dados do dataset `iris`.

Este dataset é o mais baixado do repositório de aprendizado de máquina [UCI](https://archive.ics.uci.edu/ml/), listando medidas de pétalas e sépalas de três espécies de flores de íris.

Por conveniência, vamos baixá-lo da biblioteca `seaborn`:

In [ ]:
import seaborn as sns
dados_íris = sns.load_dataset('iris')
dados_íris

Como podemos ver, o dataset contém largura e altura das sépalas e pétalas de 150 amostras de flor íris.

Vamos ver quantos exemplos temos por espécie:

In [ ]:
dados_íris['species'].value_counts()

Para agrupar este dataset por espécie, podemos usar o método `groupby()`:

In [ ]:
grupos_íris = dados_íris.groupby(['species'])

Podemos, então, tratar cada um grupo como um `DataFrame` usando o método `get_group()`:

In [ ]:
grupos_íris.get_group('versicolor').head()

O agrupamento nos permite computar estatísticas sobre os grupos ao mesmo tempo ou individualmente:

#### Ao mesmo tempo

In [ ]:
grupos_íris.min()

In [ ]:
grupos_íris.max()

In [ ]:
grupos_íris.mean()

#### Individualmente

In [ ]:
grupos_íris.get_group("versicolor").describe()

In [ ]:
grupo_versicolor = grupos_íris.get_group("versicolor")
grupo_versicolor.count()

#### Agregando por múltiplas características

Um recurso poderoso do Pandas é permitir agregações a partir de múltiplas características.

Em geral, usamos esse recurso quando temos um conjunto de dados que apresentam características categóricas e númericas.

No dataset `iris`, no entanto, temos apenas uma características categórica disponível.

Vamos aproveitar essa situação e dar uma olhada em um recurso bem legal do Pandas, chamado discretização em intervalos:

In [ ]:
pd.cut(dados_íris["petal_width"], bins=3)

Entendeu o que aconteceu? 

O método `cut()` calculou os valores máximo e mínimo para a característica `petal_width` e dividiu esse intervalo em três subintervalos.

Assim, cada um dos valores originais foi substituído pelo subintervalo ao qual ele pertecene e passamos ter uma varíavel categórica 😄

Vamos substituir os dados originais pelos dados categorizados:

In [ ]:
dados_íris["petal_width"] = pd.cut(dados_íris["petal_width"], bins=3)
dados_íris

Um recurso adicional do Pandas para lidar com características categóricas é renomear as categorias.

Vamos renomear os subintervalos gerados.

Note que desta vez estamos alterando os dados originais diretamente usando a opção `inplace=True` (quase todos os métodos Pandas aceitam essa opção).

In [ ]:
dados_íris["petal_width"].cat.rename_categories(["low", "medium", "high"], inplace=True)
dados_íris

Agora que nosso dataset apresenta duas características categóricas, podemos fazer agregações por múltiplas características:

In [ ]:
grupo2_íris = dados_íris.groupby(["species","petal_width"]).size()
grupo2_íris

Neste caso, em vez de produzirmos os grupos, produzimos diretamente a agregação usando o método `size()`, que conta o tamanho de cada grupo.

Pelos dados acima, podemos verificar que todas as flores de íris da espécie `setosa` presentes no dataset apresentam uma largura de pétala pequena.

Também é possível fazer uma excelente separação entre as espécies `versicolor` e `virginica`.

Note que os dados acima são uma série que apresentam um índice em múltiplos níveis (conhecido no Pandas como `MultiIndex`):

In [ ]:
grupo2_íris.index

Em meio às mensagens verbosas do Pandas, vemos que há dois níveis neste índice (`levels`), cujos nomes (`names`)  são `species` e `petal_width`.

Podemos indexar esta série de várias formas diferentes

In [ ]:
grupo2_íris["virginica","high"]

In [ ]:
grupo2_íris["virginica",]

In [ ]:
grupo2_íris[:,"high"]

Também podemos converter essa série em um `DataFrame`. 

Para isso, usamos o método `reset_index()` e informamos o nome que queremos dar à série:

In [ ]:
df_íris = grupo2_íris.reset_index(name="count")
df_íris

### Tabelas dinâmicas

Uma outra forma de agregação disponível no Pandas é através de tabelas dinâmicas.

Neste caso, usamos o método `pivot_table()` e devemos informar as caraterísticas para o agrupamento a nível de linhas (`index`) e de colunas (`columns`).

Também podemos informar um método de agregação usando a opção `aggfunc`, que por padrão calcula a média:

In [ ]:
pt_íris = dados_íris.pivot_table(index="species", columns="petal_width", aggfunc="size")
pt_íris

Note que a tabela dinâmica tenta gerar todas as combinações possíveis entre os valores das característica de linha e de coluna.

Como nosso dataset não apresenta observações da espécie `setosa` com largura de pétala `medium` ou `high`, esses valores são marcados como faltando/inválidos.

O método `pivot_table()` fornece a opção `fill_value`, que nos permite escolher como preencher esses casos:

In [ ]:
pt_íris = dados_íris.pivot_table(index="species", columns="petal_width", aggfunc="size", fill_value=0)
pt_íris

O método `pivot_table()` produz um objeto do tipo `DataFrame`.

Assim, a indexação funciona da maneira como já conhecemos:

In [ ]:
pt_íris.loc["versicolor"]

In [ ]:
pt_íris.loc["versicolor","low"]

In [ ]:
pt_íris.loc[:,"low"]